In [ ]:
import json
import os
from pathlib import Path
from typing import List

import tiktoken
from dotenv import load_dotenv

from discharge_summaries.openai_llm.chat_models import AzureOpenAIChatModel
from discharge_summaries.openai_llm.message import Message, Role
from discharge_summaries.openai_llm.token_count import (
    num_tokens_from_messages_azure_engine,
)
from discharge_summaries.schemas.mimic import Note

In [ ]:
load_dotenv()

In [ ]:
DATA_DIR = Path.cwd().parent / "data"

TRAINING_DATASET_PATH = DATA_DIR / "train_all_ds.pkl"
RANDOM_SEED = 23
AZURE_ENGINE = "gpt-4"
AZURE_API_VERSION = "2023-07-01-preview"
# AZURE_ENGINE = "gpt-35-turbo"
# AZURE_API_VERSION = "2023-07-01-preview"

GUIDELINES_JSON_SCHEMA_PATH = (
    Path.cwd().parent
    / "guidelines"
    / "eDischarge-Summary-v2.1-1st-Feb-21_pydantic.json"
)

INPUT_FPATH = DATA_DIR / "prsb_example.txt"
TOKENIZER = tiktoken.get_encoding("cl100k_base")

In [ ]:
output_dir = Path.cwd() / "output" / INPUT_FPATH.stem
if not output_dir.exists():
    output_dir.mkdir(parents=True)
output_dir

In [ ]:
llm = AzureOpenAIChatModel(
    api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version=AZURE_API_VERSION,
    engine=AZURE_ENGINE,
    temperature=0,
    timeout=20,
)

In [ ]:
sections_to_include = {
    "social_context",
    "participation_in_research",
    "admission_details",
    "individual_requirements",
    "diagnoses",
    "procedures",
    "clinical_summary",
    "assessment_scale",
    "allergies_and_adverse_reactions",
    "patient_and_carer_concerns_expectations_and_wishes",
    "information_and_advice_given",
    "plan_and_requested_actions",
    "safety_alerts",
}

In [ ]:
physican_notes = [
    Note(
        text=INPUT_FPATH.read_text(),
        datetime="2021-01-01 17:00:00",
        category="",
        description="",
    )
]

Remove indents (save tokens)

In [ ]:
guidelines_json_schema_json = json.loads(GUIDELINES_JSON_SCHEMA_PATH.read_text())

In [ ]:
ignored_sections = {
    section_name
    for section_name in guidelines_json_schema_json["properties"].keys()
    if section_name not in sections_to_include
}
ignored_sections

In [ ]:
guidelines_json_schema_json["properties"] = {
    k: v
    for k, v in guidelines_json_schema_json["properties"].items()
    if k in sections_to_include
}
guidelines_json_schema_str = json.dumps(guidelines_json_schema_json)

In [ ]:
(output_dir.parent / "reduced_schema.json").write_text(
    json.dumps(guidelines_json_schema_json, indent=4)
)

In [ ]:
SYSTEM_MESSAGE = Message(
    role=Role.SYSTEM,
    content=f"""You are a consultant doctor tasked with writing a patients discharge summary.
Only the information in the physician notes provided by the user can be used for this task.
Each physician note has a title of the format Physician Note [number]: [timestamp].

The discharge summary must be written in accordance with the following json schema.
{guidelines_json_schema_str}
If the information is not present to fill in a field, answer it with an empty string.
""",
)

In [ ]:
len(TOKENIZER.encode(SYSTEM_MESSAGE.content))

In [ ]:
def generate_notes_string(notes: List[Note]):
    return "\n\n".join(
        f"Physician Note {idx+1}: {note.datetime}\n{note.text}"
        for idx, note in enumerate(notes)
    )


def generate_discharge_summary(
    notes: List[Note], max_prompt_tokens=31000
) -> List[Message]:
    notes_string = generate_notes_string(notes)
    user_message_content = (
        "Generate the discharge summary json given the following physician"
        f" notes\n\n{notes_string}"
    )
    prompt_messages = [
        SYSTEM_MESSAGE,
        Message(role=Role.USER, content=user_message_content),
    ]

    num_prompt_tokens = num_tokens_from_messages_azure_engine(
        prompt_messages, AZURE_ENGINE, AZURE_API_VERSION
    )
    if num_prompt_tokens > max_prompt_tokens:
        raise ValueError(
            f"Prompt has {num_prompt_tokens} tokens, which is greater than the max of"
            f" {max_prompt_tokens}."
        )

    return prompt_messages + [llm.query(prompt_messages)]

In [ ]:
output_notes = generate_discharge_summary(physican_notes)
(output_dir / "pred.json").write_text(
    json.dumps(json.loads(output_notes[-1].content), indent=4)
)

In [ ]:
gt_fpath = INPUT_FPATH.parent / f"{INPUT_FPATH.stem}_gt.json"
gt_json = {
    section_name: v
    for section_name, v in json.loads(gt_fpath.read_text()).items()
    if section_name in sections_to_include
}
(output_dir / "gt.json").write_text(json.dumps(gt_json, indent=4))